# Binary Classification with Tweet Embeddings

In [1]:
# import modules
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.base import TransformerMixin
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import re
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer

## Load data

In [2]:
xls = pd.ExcelFile('../../../ForLastExpt.xlsx')

In [3]:
class1_train = pd.read_excel(xls, 'Class=1 Training')
class1_tune = pd.read_excel(xls, 'Class=1 Validation')
class1_test_pt1 = pd.read_excel(xls, 'Class=1 Test Part-1')
class1_test_pt2 = pd.read_excel(xls, 'Class=1 Test Part-2')
class0_train = pd.read_excel(xls, 'Class=0 Training')
class0_tune = pd.read_excel(xls, 'Class=0 Validation')
class0_test_pt1 = pd.read_excel(xls, 'Class=0 Test Part-1')
class0_test_pt2 = pd.read_excel(xls, 'Class=0 Test Part-2')

In [4]:
train = pd.concat([class1_train, class0_train], ignore_index = True)
tune = pd.concat([class1_tune, class0_tune], ignore_index = True)
test_pt1 = pd.concat([class1_test_pt1, class0_test_pt1], ignore_index = True)
test_pt2 = pd.concat([class1_test_pt2, class0_test_pt2], ignore_index = True)
test = pd.concat([test_pt1, test_pt2], ignore_index = True)
train_tune = pd.concat([train, tune], ignore_index = True)

In [5]:
class1_train

,Is_Unreliable,Category,Tweet
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government..."
2,1,"1, 4, 9",It was only a matter of time before the mainst...
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...
...,...,...,...
95,1,"11, 10",EOIR has ordered immigration court staff to re...
96,1,10,President - who said last night that #coronavi...
97,1,11,"Mashhad, NE #Iran Man collapses possibly due t..."
98,1,"10, 11",Pence caught in lies about insurance covering ...


In [6]:
train

,Is_Unreliable,Category,Tweet
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government..."
2,1,"1, 4, 9",It was only a matter of time before the mainst...
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...
...,...,...,...
195,0,NaN,Short term financial relief overseas during Co...
196,0,NaN,Coronavirus: Physician tested positive for COV...
197,0,NaN,Coronavirus: Toronto March break camps prepare...
198,0,NaN,Follow live news coverage from Italy on the CO...


## Derive text vectors from word embeddings

In [7]:
embeddings = pd.read_csv('word_embed_250.csv', index_col = 0)
embeddings

,Comp 1,Comp 2,Comp 3,Comp 4,Comp 5,Comp 6,Comp 7,Comp 8,Comp 9,Comp 10,...,Comp 241,Comp 242,Comp 243,Comp 244,Comp 245,Comp 246,Comp 247,Comp 248,Comp 249,Comp 250
!,-0.018764,0.032525,0.015407,0.009157,0.014125,0.030538,0.043033,-0.021040,-0.004332,-0.008598,...,0.010041,0.009298,0.035440,-0.000282,-0.217032,-0.135531,-0.014149,-0.002041,0.023375,0.012812
#,-0.001960,0.038517,0.010817,-0.079002,0.010372,0.080606,-0.004860,0.002473,-0.005619,-0.001717,...,-0.005194,-0.037717,0.029850,-0.002985,-0.000654,-0.010842,-0.116837,0.022335,-0.002944,0.012674
(,-0.005236,0.007737,0.004173,0.003378,0.009708,-0.197379,0.064630,-0.008035,-0.000175,0.007928,...,-0.000881,-0.011962,-0.038069,-0.006983,0.003293,-0.003806,0.024025,0.011904,-0.000179,0.012640
),-0.005141,0.009326,0.004000,0.003023,0.009742,-0.192747,0.107004,-0.008387,-0.001618,0.008576,...,-0.000223,-0.010377,-0.035645,-0.006858,0.002913,-0.002493,0.023433,0.010823,-0.000285,0.013780
",",0.005696,-0.020444,0.010301,-0.009947,-0.015616,-0.028876,0.068807,0.029133,0.008748,0.002209,...,0.007283,0.044907,0.005122,-0.045353,-0.001367,0.013397,-0.012996,-0.016318,-0.005606,-0.005031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‘,-0.004653,-0.000871,-0.004893,-0.007177,0.005050,0.013345,0.005497,0.009252,-0.004917,-0.001666,...,0.009270,-0.002894,0.001317,-0.014909,-0.002056,-0.000942,0.000207,0.021728,-0.004728,0.004189
’,-0.020318,-0.011916,-0.010054,-0.007211,-0.018799,-0.010868,-0.004023,-0.022962,0.064756,0.031455,...,-0.058205,0.035277,0.006492,0.009137,0.016892,0.062706,0.021387,-0.002337,0.006014,-0.014130
“,0.138389,0.008750,0.022160,-0.000766,-0.023548,0.028208,-0.003127,-0.011791,-0.004823,0.008236,...,0.000869,-0.001796,-0.037675,0.006247,-0.001588,-0.002131,0.013262,-0.025095,-0.006124,0.008645
”,0.174268,0.012348,0.001448,-0.005890,-0.014411,0.006477,0.000497,-0.009226,-0.003290,0.012056,...,-0.000187,-0.003730,-0.002160,0.009387,0.002483,-0.008567,0.014139,-0.018222,-0.002453,-0.000633


In [8]:
# to convert contractions picked up by word_tokenize() into full words
contractions = {
    "n't": 'not',
    "'ve": 'have',
    "'s": 'is', # note that this will include possessive nouns
    'gonna': 'going to',
    'gotta': 'got to',
    "'d": 'would',
    "'ll": 'will',
    "'re": 'are',
    "'m": 'am',
    'wanna': 'want to'
}

# to convert nltk_pos tags to wordnet-compatible PoS tags
def convert_pos_wordnet(tag):
    tag_abbr = tag[0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
                
    if tag_abbr in tag_dict:
        return tag_dict[tag_abbr]

In [9]:
def get_text_vectors(word_embeddings, # pandas dataframe with row index = vocabulary
                     text_list, # list of strings
                     remove_stopwords = True,
                     lowercase = True,
                     lemmatize = True,
                     add_start_end_tokens = True):
    
    lemmatizer = WordNetLemmatizer()
    
    for k in range(len(text_list)):
        text = text_list[k]
        text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
        text_vec = np.zeros(word_embeddings.shape[1])
        words = word_tokenize(text)
        tracker = 0 # to track whether we've encountered a word for which we have an embedding (in each tweet)
        
        if remove_stopwords:
            clean_words = []
            for word in words:
                if word.lower() not in set(stopwords.words('english')):
                    clean_words.append(word)
            words = clean_words

        if lowercase:
            clean_words = []
            for word in words:
                clean_words.append(word.lower())

            words = clean_words

        if lemmatize:
            clean_words = []
            for word in words:
                PoS_tag = pos_tag([word])[0][1]

                # to change contractions to full word form
                if word in contractions:
                    word = contractions[word]

                if PoS_tag[0].upper() in 'JNVR':
                    word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                else:
                    word = lemmatizer.lemmatize(word)

                clean_words.append(word)

            words = clean_words

        if add_start_end_tokens:
            words = ['<START>'] + words + ['<END>']
        
        for i in range(len(words)):
            word = words[i]
            if word in word_embeddings.index.tolist():
                word_embed_vec = word_embeddings.loc[word].values
                if tracker == 0:
                    text_matrix = word_embed_vec
                else:
                    text_matrix = np.vstack((text_matrix, word_embed_vec))
                    
                # only increment if we have come across a word in the embeddings dictionary
                tracker += 1
                    
        for j in range(len(text_vec)):
            text_vec[j] = text_matrix[:,j].mean()
            
        if k == 0:
            full_matrix = text_vec
        else:
            full_matrix = np.vstack((full_matrix, text_vec))
            
    return full_matrix

In [10]:
train_X = get_text_vectors(embeddings, train['Tweet'])
train_y = train['Is_Unreliable']

tune_X = get_text_vectors(embeddings, tune['Tweet'])
tune_y = tune['Is_Unreliable']

test_pt1_X = get_text_vectors(embeddings, test_pt1['Tweet'])
test_pt1_y = test_pt1['Is_Unreliable']

test_pt2_X = get_text_vectors(embeddings, test_pt2['Tweet'])
test_pt2_y = test_pt2['Is_Unreliable']

test_X = get_text_vectors(embeddings, test['Tweet'])
test_y = test['Is_Unreliable']

train_tune_X = get_text_vectors(embeddings, train_tune['Tweet'])
train_tune_y = train_tune['Is_Unreliable']

## Binary classification: two-fold CV

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

In [12]:
# SVC hyperparams to optimize
kernel = ['rbf', 'linear', 'poly', 'sigmoid']
C = [0.001, 0.01, 0.1, 1, 10]

### Fold 1

In [13]:
# training & tuning
models = []
tune_auc = []
for ker in kernel:
    for el in C:
        svc = SVC(C = el, kernel = ker, probability = True)
        svc.fit(train_X, train_y)
        models.append(svc)
        tune_predict = svc.predict_proba(tune_X)[:,1]
        auc = roc_auc_score(tune_y, tune_predict)
        tune_auc.append(auc)
        
opt_model = models[tune_auc.index(max(tune_auc))]
opt_model

SVC(C=1, probability=True)

In [14]:
tune_auc

[0.5,
 0.5,
 0.925625,
 0.9337500000000001,
 0.9237500000000001,
 0.07999999999999999,
 0.07999999999999999,
 0.07999999999999999,
 0.090625,
 0.9293750000000001,
 0.08437499999999999,
 0.5,
 0.08437499999999999,
 0.9125,
 0.905,
 0.07937499999999999,
 0.35,
 0.9181250000000001,
 0.928125,
 0.8293750000000001]

In [15]:
# testing
test_scores = dict()
test_predict = opt_model.predict(test_X)
test_predict_proba = opt_model.predict_proba(test_X)[:,1]
test_scores['auc'] = [roc_auc_score(test_y, test_predict_proba)]
test_scores['accuracy'] = [accuracy_score(test_y, test_predict)]
test_scores['recall'] = [recall_score(test_y, test_predict)]
test_scores['precision'] = [precision_score(test_y, test_predict)]
test_scores['f1'] = [f1_score(test_y, test_predict)]
test_scores

{'auc': [0.8986224489795918],
 'accuracy': [0.7821428571428571],
 'recall': [0.9571428571428572],
 'precision': [0.708994708994709],
 'f1': [0.8145896656534953]}

### Fold 2

In [16]:
# training & tuning
models = []
tune_auc = []
for ker in kernel:
    for el in C:
        svc = SVC(C = el, kernel = ker, probability = True)
        svc.fit(test_pt1_X, test_pt1_y)
        models.append(svc)
        tune_predict = svc.predict_proba(test_pt2_X)[:,1]
        auc = roc_auc_score(test_pt2_y, tune_predict)
        tune_auc.append(auc)
        
opt_model = models[tune_auc.index(max(tune_auc))]
opt_model

SVC(C=1, kernel='sigmoid', probability=True)

In [17]:
tune_auc

[0.5,
 0.5,
 0.5,
 0.894375,
 0.8887499999999999,
 0.09062500000000001,
 0.09062500000000001,
 0.09062500000000001,
 0.16593750000000002,
 0.90375,
 0.15250000000000002,
 0.15250000000000002,
 0.15250000000000002,
 0.8806250000000001,
 0.8718750000000001,
 0.08687500000000001,
 0.40375,
 0.6875,
 0.906875,
 0.820625]

In [18]:
# testing
test_predict = opt_model.predict(train_tune_X)
test_predict_proba = opt_model.predict_proba(train_tune_X)[:,1]
test_scores['auc'].append(roc_auc_score(train_tune_y, test_predict_proba))
test_scores['accuracy'].append(accuracy_score(train_tune_y, test_predict))
test_scores['recall'].append(recall_score(train_tune_y, test_predict))
test_scores['precision'].append(precision_score(train_tune_y, test_predict))
test_scores['f1'].append(f1_score(train_tune_y, test_predict))
test_scores

{'auc': [0.8986224489795918, 0.9088265306122448],
 'accuracy': [0.7821428571428571, 0.8321428571428572],
 'recall': [0.9571428571428572, 0.8214285714285714],
 'precision': [0.708994708994709, 0.8394160583941606],
 'f1': [0.8145896656534953, 0.8303249097472925]}

### Average results

In [19]:
sum(test_scores['auc'])/2

0.9037244897959182

In [20]:
sum(test_scores['accuracy'])/2

0.8071428571428572

In [21]:
sum(test_scores['recall'])/2

0.8892857142857142

In [22]:
sum(test_scores['precision'])/2

0.7742053836944347

In [23]:
sum(test_scores['f1'])/2

0.822457287700394